In [150]:
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [151]:
from tensorflow import keras

In [152]:
df=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")

In [153]:
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [155]:
df.isna().value_counts()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  free sulfur dioxide  total sulfur dioxide  density  pH     sulphates  alcohol  quality
False          False             False        False           False      False                False                 False    False  False      False    False      4898
Name: count, dtype: int64

In [156]:
df.isnull().value_counts()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  free sulfur dioxide  total sulfur dioxide  density  pH     sulphates  alcohol  quality
False          False             False        False           False      False                False                 False    False  False      False    False      4898
Name: count, dtype: int64

In [157]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,4898.0,6.854788,0.843868,3.80000,6.300000,6.80000,7.3000,14.20000
volatile acidity,4898.0,0.278241,0.100795,0.08000,0.210000,0.26000,0.3200,1.10000
citric acid,4898.0,0.334192,0.121020,0.00000,0.270000,0.32000,0.3900,1.66000
residual sugar,4898.0,6.391415,5.072058,0.60000,1.700000,5.20000,9.9000,65.80000
chlorides,4898.0,0.045772,0.021848,0.00900,0.036000,0.04300,0.0500,0.34600
free sulfur dioxide,4898.0,35.308085,17.007137,2.00000,23.000000,34.00000,46.0000,289.00000
total sulfur dioxide,4898.0,138.360657,42.498065,9.00000,108.000000,134.00000,167.0000,440.00000
density,4898.0,0.994027,0.002991,0.98711,0.991723,0.99374,0.9961,1.03898
pH,4898.0,3.188267,0.151001,2.72000,3.090000,3.18000,3.2800,3.82000
sulphates,4898.0,0.489847,0.114126,0.22000,0.410000,0.47000,0.5500,1.08000


In [158]:
quality=df['quality']
display(quality)

0       6
1       6
2       6
3       6
4       6
       ..
4893    6
4894    5
4895    6
4896    7
4897    6
Name: quality, Length: 4898, dtype: int64

In [159]:
df.drop(columns='quality',axis=1)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9
...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8


In [160]:

X_train,X_test,y_train,y_test=train_test_split(df,quality,test_size=0.25,random_state=45)

In [161]:
signature=infer_signature(X_train.to_numpy(),y_train)

/home/test-14900/anaconda3/envs/conda-env-3-11/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [162]:
### ANN
def train_model(parameters,epochs,train_x,train_y,test_x,test_y):
    ## model architecture
    mean=np.mean(test_x,axis=0)
    var=np.var(test_x,axis=0)
    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation="relu"),
            keras.layers.Dense(1)
								]
				)
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=parameters["lr"],momentum=parameters["momentum"]
				),
                loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    
				)
    with mlflow.start_run(nested=True):
            model.fit(train_x,train_y,validation_data=(test_x,test_y),epochs=epochs,batch_size=64)
            eval_result=model.evaluate(test_x,test_y,batch_size=64)
            eval_rmse=eval_result[1]
            mlflow.log_params(params=parameters)
            mlflow.log_metric("eval_rmse",eval_rmse)
            mlflow.tensorflow.log_model(model,"model",signature=signature)
            return {"loss":eval_rmse,"status":STATUS_OK,"model":model}

In [163]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")


In [164]:
def objective(params):
    result=train_model(params,50,X_train,y_train,X_test,y_test)
    return result

In [165]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [166]:
mlflow.set_experiment("ANN with mlflow")
with mlflow.start_run():
    # Conduct hyperparam search using Hyperopt
    trials=Trials()
    best=fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=5,trials=trials)
    best_run=sorted(trials.results,key=lambda x:x["loss"])[0]
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run['loss'])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)
    

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

Epoch 1/50                                           

 1/58 ━━━━━━━━━━━━━━━━━━━━ 14s 256ms/step - loss: 32.4280 - root_mean_squared_error: 5.6946
56/58 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - loss: 21.5074 - root_mean_squared_error: 4.5972 
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 13.3084 - root_mean_squared_error: 3.6481 - val_loss: 3.3794 - val_root_mean_squared_error: 1.8383

Epoch 2/50                                           

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 2.8166 - root_mean_squared_error: 1.6783
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.3493 - root_mean_squared_error: 1.5327 - val_loss: 1.6843 - val_root_mean_squared_error: 1.2978

Epoch 3/50                                           

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 2.0461 - root_mean_squared_error: 1.4304
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5273 - root_mean_squared_error: 1.2358 - val_loss: 1.2753 - val_root_mean_squared_error: 1.1293

Epoch 4/50                        

2026/01/20 09:05:47 INFO mlflow.tracking._tracking_service.client: 🏃 View run judicious-crane-149 at: http://127.0.0.1:5000/#/experiments/303954292476706878/runs/a8a5268b924d4ba89d03bcd3700662a2.

2026/01/20 09:05:47 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/303954292476706878.



Epoch 1/50                                                                     

 1/58 ━━━━━━━━━━━━━━━━━━━━ 14s 255ms/step - loss: 34.7262 - root_mean_squared_error: 5.8929
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6614 - root_mean_squared_error: 1.9135 - val_loss: 0.5570 - val_root_mean_squared_error: 0.7463

Epoch 2/50                                                                     

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3671 - root_mean_squared_error: 0.6059
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3135 - root_mean_squared_error: 0.5599 - val_loss: 0.2068 - val_root_mean_squared_error: 0.4547

Epoch 3/50                                                                     

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1455 - root_mean_squared_error: 0.3815
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1214 - root_mean_squared_error: 0.3484 - val_loss: 0.0905 - val_root_mean_squared_error: 0.3009

Epoch 4/50                                       

2026/01/20 09:05:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run colorful-panda-575 at: http://127.0.0.1:5000/#/experiments/303954292476706878/runs/ed383f645ec645f08a7b8415ae982cec.

2026/01/20 09:05:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/303954292476706878.



...

2026/01/20 09:06:20 INFO mlflow.tracking._tracking_service.client: 🏃 View run funny-grub-837 at: http://127.0.0.1:5000/#/experiments/303954292476706878/runs/f2cf8cd81b7846b2aaa9f36ed1c48f02.

2026/01/20 09:06:20 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/303954292476706878.



Epoch 1/50                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 14s 259ms/step - loss: 37.7018 - root_mean_squared_error: 6.1402
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 28.1068 - root_mean_squared_error: 5.3016 - val_loss: 21.8346 - val_root_mean_squared_error: 4.6728

Epoch 2/50                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 22.6777 - root_mean_squared_error: 4.7621
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17.8342 - root_mean_squared_error: 4.2231 - val_loss: 13.7410 - val_root_mean_squared_error: 3.7069

Epoch 3/50                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 12.3555 - root_mean_squared_error: 3.5150
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11.2174 - root_mean_squared_error: 3.3492 - val_loss: 8.5846 - val_root_mean_squared_error: 2.9300

Epoch 4/50                          

2026/01/20 09:06:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run powerful-cod-963 at: http://127.0.0.1:5000/#/experiments/303954292476706878/runs/4bad61ea97ff450bafffb745d898fdde.

2026/01/20 09:06:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/303954292476706878.



100%|██████████| 5/5 [00:54<00:00, 10.88s/trial, best loss: 0.024224547669291496]


2026/01/20 09:06:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run respected-pug-879 at: http://127.0.0.1:5000/#/experiments/303954292476706878/runs/81f7edb13a2248fa923a2c7a80f14aed.
2026/01/20 09:06:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/303954292476706878.


In [167]:
## validating model
import mlflow.pyfunc

model_uri = 'runs:/6d4ec9c4d2244eeb8dba78b599b86e45/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [168]:
from mlflow.models import convert_input_example_to_serving_input

In [169]:
serving_payload = convert_input_example_to_serving_input(X_test)

In [170]:
from mlflow.models import validate_serving_input

In [171]:
validate_serving_input(model_uri,serving_payload)

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[5.003921 ],
       [5.9681168],
       [6.0304794],
       ...,
       [4.9877253],
       [5.997781 ],
       [6.0241003]], dtype=float32)